# Soliton Validation: Sign Optimization Benchmark

This notebook validates all sign-optimization approaches on liquid crystal director fields.
It provides a controlled benchmark by:

1. Loading a director field (real LCSim data or synthetic cholesteric)
2. Randomly scrambling 50% of voxel signs (known ground truth)
3. Running all 6 sign-optimization methods
4. Comparing: energy recovery, sign accuracy, timing
5. Visualizing results: director slices, error maps, Frank energy decomposition

**Optimizers tested:**
- Combined (V1 chain propagation + iterative flip)
- Layer Propagation (layer-by-layer + refinement)
- Graph Cuts (min-cut/max-flow on neighborhood graph)
- Simulated Annealing (Metropolis-Hastings with adaptive temperature)
- Hierarchical (multi-scale coarse-to-fine)
- Belief Propagation (message passing on factor graph)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import fcpm
from fcpm.reconstruction.base import OptimizationResult
from fcpm.reconstruction.optimizers import (
    CombinedOptimizer,
    LayerPropagationOptimizer,
    GraphCutsOptimizer,
    SimulatedAnnealingOptimizer,
    SimulatedAnnealingConfig,
    HierarchicalOptimizer,
    BeliefPropagationOptimizer,
    BeliefPropagationConfig,
)

print(f"FCPM version: {fcpm.__version__}")

## 1. Load Director Field

If you have real LCSim data (e.g. cholesteric fingers, Z-solitons, torons from article.lcpen),
set `DATA_PATH` below. Otherwise, a synthetic cholesteric is used as fallback.

In [ ]:
# Set to your LCSim NPZ file path, or None for synthetic
DATA_PATH = None  # e.g. '../data/CF1.npz'
SEED = 42

if DATA_PATH is not None:
    from pathlib import Path
    if Path(DATA_PATH).exists():
        director_gt, settings = fcpm.load_lcsim_npz(DATA_PATH)
        print(f"Loaded LCSim data from {DATA_PATH}")
        print(f"Shape: {director_gt.shape}")
        if settings:
            print(f"Settings: {settings}")
    else:
        print(f"File not found: {DATA_PATH}, using synthetic fallback")
        DATA_PATH = None

if DATA_PATH is None:
    director_gt = fcpm.create_cholesteric_director(shape=(64, 64, 32), pitch=8.0)
    print(f"Created synthetic cholesteric: shape={director_gt.shape}, pitch=8.0")

In [ ]:
# Visualize ground truth
z_mid = director_gt.shape[2] // 2
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

fcpm.plot_director_slice(director_gt, z_idx=0, step=2, ax=axes[0],
                         title=f'Ground Truth (z=0)')
fcpm.plot_director_slice(director_gt, z_idx=z_mid, step=2, ax=axes[1],
                         title=f'Ground Truth (z={z_mid})')
fcpm.plot_director_slice(director_gt, z_idx=director_gt.shape[2]-1, step=2, ax=axes[2],
                         title=f'Ground Truth (z={director_gt.shape[2]-1})')
plt.tight_layout()
plt.show()

## 2. Scramble Signs (Controlled Test)

Randomly flip the sign of 50% of voxels. Since we know the ground truth,
we can measure both energy recovery and sign accuracy.

In [ ]:
def scramble_signs(director, seed=42):
    """Randomly flip signs of 50% of voxels."""
    rng = np.random.default_rng(seed)
    n = director.to_array().copy()
    mask = rng.random(n.shape[:3]) < 0.5
    n[mask] = -n[mask]
    return fcpm.DirectorField.from_array(n, metadata=director.metadata)

director_scrambled = scramble_signs(director_gt, seed=SEED)

gt_energy = fcpm.compute_gradient_energy(director_gt)
scrambled_energy = fcpm.compute_gradient_energy(director_scrambled)

print(f"Ground truth energy:  {gt_energy:.2f}")
print(f"Scrambled energy:     {scrambled_energy:.2f}")
print(f"Energy increase:      {scrambled_energy - gt_energy:.2f} "
      f"({100*(scrambled_energy - gt_energy)/gt_energy:.1f}%)")

In [ ]:
# Visualize scrambled vs ground truth
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fcpm.plot_director_slice(director_gt, z_idx=z_mid, step=2, ax=axes[0],
                         title='Ground Truth')
fcpm.plot_director_slice(director_scrambled, z_idx=z_mid, step=2, ax=axes[1],
                         title='Scrambled (50% flipped)')
plt.tight_layout()
plt.show()

## 3. Run All Optimizers

Each optimizer receives the same scrambled director field and attempts to recover
consistent signs. We track energy reduction, sign accuracy, and wall-clock time.

In [ ]:
optimizers = [
    ("Combined (V1)", CombinedOptimizer()),
    ("LayerPropagation", LayerPropagationOptimizer()),
    ("GraphCuts", GraphCutsOptimizer()),
    ("SimulatedAnnealing", SimulatedAnnealingOptimizer(
        SimulatedAnnealingConfig(max_iterations=10000, seed=SEED))),
    ("Hierarchical", HierarchicalOptimizer()),
    ("BeliefPropagation", BeliefPropagationOptimizer(
        BeliefPropagationConfig(max_iterations=30))),
]

results = []
result_directors = {}  # Store optimized directors for visualization

for name, optimizer in optimizers:
    print(f"\n--- {name} ---")
    t0 = time.perf_counter()
    result: OptimizationResult = optimizer.optimize(director_scrambled, verbose=True)
    elapsed = time.perf_counter() - t0

    sign_acc = fcpm.sign_accuracy(result.director, director_gt)

    energy_gap = scrambled_energy - gt_energy
    recovered = scrambled_energy - result.final_energy
    recovery_pct = 100.0 * recovered / energy_gap if energy_gap > 0 else 100.0

    metrics = {
        "method": name,
        "initial_energy": result.initial_energy,
        "final_energy": result.final_energy,
        "energy_reduction_pct": result.energy_reduction_pct,
        "sign_accuracy": sign_acc,
        "energy_recovery_pct": round(recovery_pct, 2),
        "total_flips": result.total_flips,
        "time_s": round(elapsed, 3),
    }
    results.append(metrics)
    result_directors[name] = result.director

    print(f"  Energy: {metrics['initial_energy']:.1f} -> {metrics['final_energy']:.1f} "
          f"({metrics['energy_reduction_pct']:.1f}% reduction)")
    print(f"  Sign accuracy: {sign_acc:.3f}")
    print(f"  Energy recovery: {recovery_pct:.1f}%")
    print(f"  Time: {elapsed:.3f}s")

## 4. Summary Table

In [ ]:
print(f"{'Method':<25} {'Energy Red%':>10} {'Sign Acc':>10} {'E Recovery%':>12} {'Time(s)':>8}")
print("-" * 70)
for r in results:
    print(f"{r['method']:<25} {r['energy_reduction_pct']:>9.1f}% "
          f"{r['sign_accuracy']:>9.3f} "
          f"{r['energy_recovery_pct']:>11.1f}% "
          f"{r['time_s']:>7.3f}")

## 5. Method Comparison (Bar Charts)

In [ ]:
methods = [r['method'] for r in results]
x = np.arange(len(methods))

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Energy reduction
axes[0].bar(x, [r['energy_reduction_pct'] for r in results], color='steelblue')
axes[0].set_xticks(x)
axes[0].set_xticklabels(methods, rotation=45, ha='right', fontsize=8)
axes[0].set_ylabel('Energy Reduction (%)')
axes[0].set_title('Energy Reduction')

# Sign accuracy
axes[1].bar(x, [r['sign_accuracy'] for r in results], color='forestgreen')
axes[1].set_xticks(x)
axes[1].set_xticklabels(methods, rotation=45, ha='right', fontsize=8)
axes[1].set_ylabel('Sign Accuracy')
axes[1].set_title('Sign Accuracy')
axes[1].set_ylim(0, 1.05)

# Timing
axes[2].bar(x, [r['time_s'] for r in results], color='coral')
axes[2].set_xticks(x)
axes[2].set_xticklabels(methods, rotation=45, ha='right', fontsize=8)
axes[2].set_ylabel('Time (s)')
axes[2].set_title('Execution Time')

plt.tight_layout()
plt.show()

## 6. Director Slice Comparison

Visual comparison of each optimizer's output against the ground truth.

In [ ]:
n_methods = len(result_directors)
fig, axes = plt.subplots(2, (n_methods + 2) // 2, figsize=(5 * ((n_methods + 2) // 2), 10))
axes = axes.flatten()

# Ground truth and scrambled
fcpm.plot_director_slice(director_gt, z_idx=z_mid, step=2, ax=axes[0],
                         title='Ground Truth')
fcpm.plot_director_slice(director_scrambled, z_idx=z_mid, step=2, ax=axes[1],
                         title='Scrambled')

# Each optimizer result
for i, (name, d) in enumerate(result_directors.items()):
    r = results[i]
    fcpm.plot_director_slice(d, z_idx=z_mid, step=2, ax=axes[i + 2],
                             title=f"{name}\nacc={r['sign_accuracy']:.3f}")

# Hide unused axes
for j in range(n_methods + 2, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

## 7. Error Maps

Per-voxel angular error relative to the ground truth for each method.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (name, d) in enumerate(result_directors.items()):
    fcpm.plot_error_map(d, director_gt, z_idx=z_mid, ax=axes[i])
    axes[i].set_title(f"{name}")

plt.tight_layout()
plt.show()

## 8. Frank Energy Decomposition (Splay/Twist/Bend)

Decompose the elastic energy into splay, twist, and bend contributions
for the ground truth and the best optimizer result.

In [ ]:
from fcpm.reconstruction.energy import FrankConstants, compute_frank_energy_anisotropic

# Use article.lcpen constants for 5CB
frank = FrankConstants(K1=10.3, K2=7.4, K3=16.48)

# Ground truth energy decomposition
gt_frank = compute_frank_energy_anisotropic(director_gt.to_array(), frank)
print("Ground Truth Frank Energy:")
print(f"  Splay:  {gt_frank['splay_integrated']:.2f}")
print(f"  Twist:  {gt_frank['twist_integrated']:.2f}")
print(f"  Bend:   {gt_frank['bend_integrated']:.2f}")
print(f"  Total:  {gt_frank['total_integrated']:.2f}")

In [ ]:
# Find best method by sign accuracy
best_idx = max(range(len(results)), key=lambda i: results[i]['sign_accuracy'])
best_name = results[best_idx]['method']
best_director = result_directors[best_name]

best_frank = compute_frank_energy_anisotropic(best_director.to_array(), frank)
print(f"Best Method: {best_name}")
print(f"  Splay:  {best_frank['splay_integrated']:.2f}")
print(f"  Twist:  {best_frank['twist_integrated']:.2f}")
print(f"  Bend:   {best_frank['bend_integrated']:.2f}")
print(f"  Total:  {best_frank['total_integrated']:.2f}")

In [ ]:
# Bar chart comparing Frank energy components
fig, ax = plt.subplots(figsize=(8, 5))

components = ['Splay', 'Twist', 'Bend']
gt_vals = [gt_frank['splay_integrated'], gt_frank['twist_integrated'],
           gt_frank['bend_integrated']]
best_vals = [best_frank['splay_integrated'], best_frank['twist_integrated'],
             best_frank['bend_integrated']]

x = np.arange(len(components))
width = 0.35

ax.bar(x - width/2, gt_vals, width, label='Ground Truth', color='steelblue')
ax.bar(x + width/2, best_vals, width, label=f'{best_name}', color='coral')

ax.set_xlabel('Energy Component')
ax.set_ylabel('Integrated Energy')
ax.set_title('Frank Energy Decomposition')
ax.set_xticks(x)
ax.set_xticklabels(components)
ax.legend()
plt.tight_layout()
plt.show()

## 9. Noise Sensitivity on Soliton Structures

Test how each optimizer handles sign recovery after noise has been added to the
FCPM simulation-reconstruction pipeline.

In [ ]:
noise_levels = [0.01, 0.03, 0.05, 0.10]
noise_results = {name: [] for name, _ in optimizers}

# Simulate clean FCPM
I_clean = fcpm.simulate_fcpm(director_gt)

for noise_sigma in noise_levels:
    print(f"\nNoise level: {noise_sigma*100:.0f}%")

    # Add noise and reconstruct (introduces real sign errors)
    I_noisy = fcpm.add_fcpm_realistic_noise(
        I_clean, noise_model='gaussian', gaussian_sigma=noise_sigma, seed=SEED)
    I_noisy = fcpm.normalize_fcpm(I_noisy)
    director_recon, _ = fcpm.reconstruct(I_noisy, fix_signs=False, verbose=False)

    for name, optimizer in optimizers:
        result = optimizer.optimize(director_recon, verbose=False)
        acc = fcpm.sign_accuracy(result.director, director_gt)
        noise_results[name].append(acc)
        print(f"  {name:<25} sign_accuracy={acc:.3f}")

In [ ]:
# Plot noise sensitivity
fig, ax = plt.subplots(figsize=(10, 6))
noise_pct = [n * 100 for n in noise_levels]

colors = plt.cm.Set2(np.linspace(0, 1, len(optimizers)))
for i, (name, _) in enumerate(optimizers):
    ax.plot(noise_pct, noise_results[name], 'o-', label=name,
            color=colors[i], linewidth=2, markersize=6)

ax.set_xlabel('Noise Level (%)', fontsize=12)
ax.set_ylabel('Sign Accuracy', fontsize=12)
ax.set_title('Sign Recovery vs Noise Level', fontsize=14)
ax.set_ylim(0.4, 1.05)
ax.legend(fontsize=9, loc='lower left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Spatial Error Distribution (Per-Layer Analysis)

In [ ]:
# Per-layer error for the best method
dist = fcpm.spatial_error_distribution(best_director, director_gt)

z_layers = np.arange(len(dist['layer_mean']))

fig, ax = plt.subplots(figsize=(10, 5))
ax.fill_between(z_layers, 0, dist['layer_max'], alpha=0.2, color='red', label='Max')
ax.fill_between(z_layers, 0, dist['layer_mean'], alpha=0.3, color='steelblue', label='Mean')
ax.plot(z_layers, dist['layer_median'], 'g-', linewidth=2, label='Median')

ax.set_xlabel('Z Layer', fontsize=12)
ax.set_ylabel('Angular Error (degrees)', fontsize=12)
ax.set_title(f'Error Distribution by Depth ({best_name})', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

Key observations:
- **Graph Cuts** typically achieves the highest sign accuracy on clean data (global optimum for binary labeling)
- **Hierarchical** provides a good balance of speed and accuracy
- **Combined (V1)** remains a reliable baseline
- **Simulated Annealing** can achieve excellent results but requires more time
- **Belief Propagation** is experimental and may not converge on all structures

For real-world use, we recommend starting with **Graph Cuts** or **Hierarchical** for most applications.